In [3]:
import random
import itertools

In [2]:
def draw_board(board):
  print("-----------------")
  for row in board:
    print(' '.join(map(str, row)))
  print("-----------------")

def compress_board(board):
    W, H = len(board), len(board[0])
    seq = []
    for x in range(W):
        for y in range(H):
            if board[x][y] != 0:
                seq.append(board[x][y])
    jseq = ','.join(map(str, seq))
    print("Board{W: %d, H: %d, b: []int{%s}}" % (W, H, jseq))

In [5]:
def s10req(n, s, m):
    if n == 0 and s == 0:
        yield []
        return
    if m > s or n == 0 or s < 0:
        return
    for p in range(m, min(9, s) + 1):
        for seq in s10req(n - 1, s - p, p):
            yield [p] + seq
    
sum_10_seq = {
    i: list(s10req(i, 10, 1)) for i in range(2, 5)
}   

In [6]:
def find_glades(board):
    W, H = len(board), len(board[0])
    # TODO: do better
    def _is_glade(lx, ly, rx, ry):
        if lx == rx and ly == ry:
            return False
        for x in range(lx, rx+1):
            for y in range(ly, ry+1):
                if board[x][y] != 0:
                    return False
        return True
    
    for lx in range(W):
        for ly in range(H):
            for rx in range(lx, W):
                for ry in range(ly, H):
                    if _is_glade(lx, ly, rx, ry):
                        yield (lx, ly, rx, ry)


MAX_NUMS_PER_GLADE = 4
def make_board(w, h, seed, print_board=False):
    random.seed(seed)
    board = [[0 for _ in range(h)] for _ in range(w)]
    
    def _rec():
        if all(board[x][y] != 0 for x in range(w) for y in range(h)):
            return True
        # TODO: check for isolated cells, easy pruning
        glades = list(find_glades(board))      
        random.shuffle(glades)

        for lx, ly, rx, ry in glades:
          gw, gh = rx-lx+1, ry-ly+1
          assert gw*gh >= 2
          all_xy = [(x, y) for x in range(lx, rx+1) for y in range(ly, ry+1)]

          nns = list(reversed(range(2, min(MAX_NUMS_PER_GLADE, gw*gh)+1)))
          
          for nn in nns:
            nums = random.sample(sum_10_seq[nn], 1)[0][:]
            random.shuffle(nums)
            for nxy in itertools.combinations(all_xy, nn):
                for n, (x, y) in zip(nums, nxy):
                    board[x][y] = n
                if _rec():
                    if print_board:
                        dbgboard = [[0 for _ in range(h)] for _ in range(w)]
                        for x, y in nxy:
                            dbgboard[x][y] = board[x][y]
                        draw_board(dbgboard)
                    return True
                for x, y in nxy:
                    board[x][y] = 0
        return False
    
    if not _rec():
        return None
    return board    

board = make_board(5, 5, 35, print_board=True)
if board is not None:
    draw_board(board)
    compress_board(board)



-----------------
0 0 0 0 0
0 0 0 4 6
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
-----------------
-----------------
5 5 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
-----------------
-----------------
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 5 0
0 0 0 5 0
-----------------
-----------------
0 0 0 0 0
0 2 0 0 0
0 6 0 0 0
0 2 0 0 0
0 0 0 0 0
-----------------
-----------------
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 6
0 0 0 0 4
-----------------
-----------------
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 3 7 0 0
-----------------
-----------------
0 0 0 0 0
0 0 0 0 0
0 0 1 1 1
0 0 7 0 0
0 0 0 0 0
-----------------
-----------------
0 0 0 0 0
5 0 0 0 0
1 0 0 0 0
1 0 0 0 0
3 0 0 0 0
-----------------
-----------------
0 0 3 1 3
0 0 3 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
-----------------
-----------------
5 5 3 1 3
5 2 3 4 6
1 6 1 1 1
1 2 7 5 6
3 3 7 5 4
-----------------
Board{W: 5, H: 5, b: []int{5,5,3,1,3,5,2,3,4,6,1,6,1,1,1,1,2,7,5,6,3,3,7,5,4}}
